In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
from shutil import copyfile,copy,copy2
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import keras
#%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Conv2D,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.models import Model

In [ ]:
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
img_file = '../input/dog-breed-identification/train/'

df = labels.assign(img_path=lambda x: img_file + x['id'] +'.jpg')
df.head()

In [ ]:
#Top 20 breeds in the dataset
df.breed.value_counts().sort_values(ascending=False).head(20)

In [ ]:
top_20=list(df.breed.value_counts()[0:20].index)
top_20

In [ ]:
%%time

images = np.array([img_to_array(load_img(img, target_size=(50,50,3))) for img in df['img_path'].values.tolist()])

images.shape

In [ ]:
ytrain = df.breed
ytrain = pd.get_dummies(ytrain)
ytrain.head()

In [ ]:
xtrain=images
ytrain=ytrain.values
print(xtrain.shape)
print(ytrain.shape)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
trainingdata = train_datagen.flow(xtrain,y=ytrain,batch_size=5000)

In [ ]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(50,50,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dense(120,activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(xtrain,ytrain,validation_split=0.1,verbose=1,epochs=2,batch_size=1000)

In [ ]:
plt.plot(history.history['loss'],label='Training Loss')
plt.plot(history.history['val_loss'],label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],label='Training Accuracy')
plt.plot(history.history['val_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
training_accuracy = history.history['accuracy'][-1:]
validation_accuracy = history.history['val_accuracy'][-1:]
training_loss = history.history['loss'][-1:]
validation_loss = history.history['val_loss'][-1:]
print('Training Accuracy: ',training_accuracy)
print('Validation Accuracy: ',validation_accuracy)
print('Training LOSS: ',training_loss)
print('Validation LOSS: ',validation_loss)

In [ ]:
path = '../input/'
os.listdir(path)

In [ ]:
os.chdir("/kaggle/working/")

In [ ]:
os.makedirs('/kaggle/working/temp/')

In [ ]:
from distutils.dir_util import copy_tree

fromdirectory = '../input/dog-breed-identification/'
todirectory = 'temp/'

copy_tree(fromdirectory,todirectory)

In [ ]:
import shutil
shutil.rmtree("/kaggle/working/temp/train")

In [ ]:
direc = '/kaggle/working/temp/'
test_datagen=ImageDataGenerator(rescale=1./255)
testdata = test_datagen.flow_from_directory(directory=direc,target_size=(50,50))

In [ ]:
submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
submission.head()

In [ ]:
preds = model.predict(testdata, verbose=1)

In [ ]:
resnet_5_submission = pd.DataFrame(preds)
#df = pd.DataFrame(data)
ytrain=pd.DataFrame(ytrain)
col_names = ytrain.columns.values
resnet_5_submission.columns = col_names
resnet_5_submission.insert(0, 'id', submission['id'])
resnet_5_submission.head(5)

In [ ]:
resnet_5_submission.to_csv("resnet_5_submission.csv",index=False)
pd.read_csv('resnet_5_submission.csv')